<a href="https://colab.research.google.com/github/KaterynaSiniuta/ML-Hillel/blob/main/HW_11_Text_Non_vector_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Підключаємо Google Drive з датасетами
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**ДЗ 11. Текст. Невекторні методи**

Датасет

https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Класифікація на позитивні та негативні відгуки

Використати sci-kit learn (BOW, TF_IDF)

In [ ]:
# Імпортуємо всі необхідні бібліотеки
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import sklearn

sns.set()

%config InlineBackend.figure_format = "png"

# зчитуємо набір даних
datasets_path = "/content/drive/MyDrive/Colab Notebooks/archive/"
df = pd.read_csv(datasets_path + "IMDB Dataset.csv", sep=",")


In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
# Використаємо LabelEncoder для енкодінгу таргета
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df["sentiment"] = le.fit_transform(df["sentiment"])
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
# Display the counts of each sentiment value
print(df['sentiment'].value_counts())


sentiment
1    25000
0    25000
Name: count, dtype: int64


це збалансований по кількості датасет

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
stop_words = set(ENGLISH_STOP_WORDS)
df["review"] = df["review"].apply(lambda x: " ".join(term for term in x.split() if term not in stop_words))

In [ ]:
df.head()

,review,sentiment
0,One reviewers mentioned watching just 1 Oz epi...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought wonderful way spend time hot summer ...,1
3,Basically there's family little boy (Jake) thi...,0
4,"Petter Mattei's ""Love Time Money"" visually stu...",1


In [ ]:
# Та приберемо HTML теги
import re
df["review"] = df["review"].apply(lambda x: re.compile(r'<[^>]+>').sub("", x))

In [ ]:
df.head()

,review,sentiment
0,One reviewers mentioned watching just 1 Oz epi...,1
1,A wonderful little production. The filming tec...,1
2,I thought wonderful way spend time hot summer ...,1
3,Basically there's family little boy (Jake) thi...,0
4,"Petter Mattei's ""Love Time Money"" visually stu...",1


In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk

# Завантажимо необхідні ресурси для лематизації
nltk.download('wordnet')
nltk.download('omw-1.4')

# Ініціалізація лематизатора
lemmatizer = WordNetLemmatizer()

# Функція для лематизації з визначенням частини мови
def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(word, wordnet.VERB) for word in text.split()])

# Створимо копію датафрейму (щоб не змінювати оригінал)
df_lem = df.copy(deep=True)

# Застосовуємо лематизацію до стовпця "review"
df_lem["review"] = df["review"].apply(lemmatize_text)

# Перевіримо результат
print(df_lem["review"].head())


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


0    One reviewers mention watch just 1 Oz episode ...
1    A wonderful little production. The film techni...
2    I think wonderful way spend time hot summer we...
3    Basically there's family little boy (Jake) thi...
4    Petter Mattei's "Love Time Money" visually stu...
Name: review, dtype: object


BAG OS WORDS

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, fbeta_score

# Векторизация текста
X = CountVectorizer().fit_transform(df_lem["review"])
y = df_lem["sentiment"]

# Масштабирование признаков
X = MaxAbsScaler().fit_transform(X)

# Разделение данных
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=42, stratify=y)

# Обучение модели логистической регрессии
model = LogisticRegression(random_state=42).fit(x_train, y_train)

# Оценка модели
predictions = model.predict(x_test)

print(f"Ср. квадратичная ошибка: {mean_squared_error(y_test, predictions)}")
print(f"Точность модели: {model.score(x_test, y_test) * 100:.2f}%")
print(f"F2-мера: {fbeta_score(y_test, predictions, beta=2):.4f}")


Ср. квадратичная ошибка: 0.10672
Точность модели: 89.33%
F2-мера: 0.8990


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# K-Nearest Neighbors (KNN)
knn_model = KNeighborsClassifier(n_neighbors=5).fit(x_train, y_train)
knn_predictions = knn_model.predict(x_test)

print("K-Nearest Neighbors (KNN):")
print(f"Ср. квадратичная ошибка: {mean_squared_error(y_test, knn_predictions)}")
print(f"Точность: {knn_model.score(x_test, y_test) * 100:.2f}%")
print(f"F2-мера: {fbeta_score(y_test, knn_predictions, beta=2):.4f}")


K-Nearest Neighbors (KNN):
Ср. квадратичная ошибка: 0.38584
Точность: 61.42%
F2-мера: 0.7023


Лог регресія та knn дають непогані результати

**TF-IDF**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Використовуємо TF-IDF для перетворення тексту в числовий вигляд
X_tfidf = TfidfVectorizer().fit_transform(df_lem["review"])
y = df_lem["sentiment"]

# Масштабуємо ознаки
X_tfidf = MaxAbsScaler().fit_transform(X_tfidf)

# Розділяємо дані на навчальний і тестовий набори (з урахуванням стратифікації)
x_train, x_test, y_train, y_test = train_test_split(X_tfidf, y, train_size=0.75, random_state=42, stratify=y)

# Логістична регресія (TF-IDF)
logreg_model_tfidf = LogisticRegression(random_state=42).fit(x_train, y_train)
logreg_predictions_tfidf = logreg_model_tfidf.predict(x_test)

print("Логістична регресія (TF-IDF):")
print(f"Середньоквадратична похибка: {mean_squared_error(y_test, logreg_predictions_tfidf)}")
print(f"Точність: {logreg_model_tfidf.score(x_test, y_test) * 100:.2f}%")
print(f"F2-міра: {fbeta_score(y_test, logreg_predictions_tfidf, beta=2):.4f}\n")

# Метод найближчих сусідів (KNN) (TF-IDF)
knn_model_tfidf = KNeighborsClassifier(n_neighbors=5).fit(x_train, y_train)
knn_predictions_tfidf = knn_model_tfidf.predict(x_test)

print("Метод K-найближчих сусідів (TF-IDF):")
print(f"Середньоквадратична похибка: {mean_squared_error(y_test, knn_predictions_tfidf)}")
print(f"Точність: {knn_model_tfidf.score(x_test, y_test) * 100:.2f}%")
print(f"F2-міра: {fbeta_score(y_test, knn_predictions_tfidf, beta=2):.4f}")

Логістична регресія (TF-IDF):
Середньоквадратична похибка: 0.10368
Точність: 89.63%
F2-міра: 0.9017

Метод K-найближчих сусідів (TF-IDF):
Середньоквадратична похибка: 0.4996
Точність: 50.04%
F2-міра: 0.8325


Точність KNN при it-idf нижча на 10%